In [1]:
import os
import logging
import pandas as pd
import dtale
import numpy as np
from sklearn.feature_selection import chi2, VarianceThreshold, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import eli5
import util

logging.getLogger().setLevel(logging.INFO)
np.random.seed(77213)

In [2]:
prefix = r'C:\Users\lerna\git\zhongguoyuyan\csv\dialect'
location = pd.read_csv(os.path.join(prefix, 'location.csv'), index_col=0)
homophones = pd.read_csv('homophone.csv', index_col='id', dtype={'id': str})

In [3]:
train_data, test_data = train_test_split(homophones, test_size=0.2)
train_dialect = util.get_dialect(location.loc[train_data.index])
test_dialect = util.get_dialect(location.loc[test_data.index])

In [4]:
idx = np.argsort(-VarianceThreshold(0).fit(train_data).variances_)[:100]
df = pd.DataFrame(train_data.iloc[:, idx], index=train_data.index, columns=train_data.columns)
df.insert(0, 'dialect', train_dialect)
dtale.show(df, name='train variance', ignore_duplicate=True)

In [5]:
selector = SelectKBest(chi2, k=1000).fit(train_data[train_dialect != ''], train_dialect[train_dialect != ''])
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_data[train_dialect != ''].iloc[:, idx],
    index=train_data[train_dialect != ''].index,
    columns=train_data.columns[idx]
)
df.insert(0, 'dialect', train_dialect[train_dialect != ''])
dtale.show(df, name='train chi2')

In [6]:
lr = LogisticRegression(penalty='l1', solver='saga', fit_intercept=False).fit(train_data[train_dialect != ''], train_dialect[train_dialect != ''])
print(classification_report(train_dialect[train_dialect != ''], lr.predict(train_data[train_dialect != ''])))
print(classification_report(test_dialect[test_dialect != ''], lr.predict(test_data[test_dialect != ''])))

C:\Users\lerna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

C:\Users\lerna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\lerna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\lerna\AppData\Lo

              precision    recall  f1-score   support

        东北官话       0.84      1.00      0.92        27
        中原官话       0.97      0.94      0.95        62
        兰银官话       1.00      1.00      1.00        11
        冀鲁官话       0.96      1.00      0.98        23
        北京官话       1.00      1.00      1.00        11
         吴方言       1.00      1.00      1.00        47
        客家方言       0.97      0.94      0.96        34
          平话       0.78      0.78      0.78         9
         徽方言       1.00      1.00      1.00        11
         晋方言       0.97      0.97      0.97        35
        江淮官话       1.00      0.97      0.98        33
        湖南土话       0.00      0.00      0.00         1
         湘方言       0.93      0.96      0.95        28
         粤方言       0.95      1.00      0.97        38
        胶辽官话       1.00      0.64      0.78        14
        西南官话       0.99      1.00      0.99       147
         赣方言       0.98      1.00      0.99        57
         闽方言       1.00    

In [7]:
eli5.explain_weights(lr, top=10, feature_names=train_data.columns.values)

Explanation(estimator="LogisticRegression(fit_intercept=False, penalty='l1', solver='saga')", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='东北官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容=折格隔_河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容=吉竹菊足烛_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓=折格隔_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓=吉竹菊足烛_斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停=折格隔_斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停=吉竹菊足烛_折格隔=螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙_折格隔=茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷_螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙=吉竹菊足烛_吉竹菊足烛=茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷', weight=0.5105326384685251, std=None, value=None), FeatureWeight(feature='finals_买卖矮=拆窄摘_胎台袋来菜财该改开海爱盖害=拆窄摘_拆窄摘=带拜排埋摆派牌柴晒败', weight=0.4504086270375304, std=None, value=None), FeatureWeight(feature='finals_买卖矮=百拍白麦_胎台袋来菜财该改开海爱盖害=百拍白麦_百拍白麦=带拜排埋摆派牌柴晒败', weight=0.3595478114549509, std=None, value=None), FeatureWeight(feature='tone_左可躲锁果火把假哑姐写谱赌土古苦虎所鼠举许府取数主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝早草嫂好饱找小抖走狗口酒手九感胆毯敢减险点品铲产板剪浅扁典显短碗选卷反紧隐本滚笋准粉党抢想厂响纺绑讲等肯打省梗影饼井整顶挺懂桶孔统肿=笔骨北得谷福曲_左可躲锁果火把假哑姐写谱赌土古苦虎所鼠举许府取数主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝早草嫂好饱找小抖走狗口酒手九感胆毯敢减险点品铲产板剪浅扁典显短碗选卷反紧隐本滚笋准粉党抢想厂响纺绑讲等肯打省梗影饼井整顶挺懂桶孔统肿=铁雪血脚百客尺_左可躲锁果火把假哑姐写谱赌土古苦虎所鼠举许府取数主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝早草嫂好饱找小抖走狗口酒手九感胆毯敢减险点品铲产板剪浅扁典显短碗选卷反紧隐本滚笋准粉党抢想厂响纺绑讲等肯打省梗影饼井整顶挺懂桶孔统肿=甲法_左可躲锁果火把假哑姐写谱赌土古苦虎所鼠举许府取数主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝早草嫂好饱找小抖走狗口酒手九感胆毯敢减险点品铲产板剪浅扁典显短碗选卷反紧隐本滚笋准粉党抢想厂响纺绑讲等肯打省梗影饼井整顶挺懂桶孔统肿=塔渴_女武米李脑染眼晚引=笔骨北得谷福曲_女武米李脑染眼晚引=铁雪血脚百客尺_笔骨北得谷福曲=马野吕买尾母满暖软两猛冷领_笔骨北得谷福曲=瓦雨耳老藕远网_铁雪血脚百客尺=马野吕买尾母满暖软两猛冷领_铁雪血脚百客尺=瓦雨耳老藕远网', weight=0.34614375488730714, std=None, value=None), FeatureWeight(feature='tone_河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容=夹_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓=夹_斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停=夹_夹=螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙_夹=茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷', weight=0.33570126703076913, std=None, value=None), FeatureWeight(feature='finals_折撤舌设热=破婆磨磨摸_破婆磨磨摸=车蛇射_车蛇射=拨泼末', weight=0.2859003456636228, std=None, value=None), FeatureWeight(feature='finals_破婆磨磨摸=鹅饿', weight=0.28177751338841467, std=None, value=None)], neg=[FeatureWeight(feature='tone_接贴切泼脱缺踢吃=铁雪血脚百客尺_搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆=铁雪血脚百客尺', weight=-0.7111900584675745, std=None, value=None), FeatureWeight(feature='initial_戒街解间角=戒街解间角', weight=-0.37260351773976297, std=None, value=None), FeatureWeight(feature='tone_刻侧测=接贴切泼脱缺踢吃_刻侧测=搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆_刻侧测=铁雪血脚百客尺_接贴切泼脱缺踢吃=笔骨北得谷福曲_接贴切泼脱缺踢吃=吉竹菊足烛_接贴切泼脱缺踢吃=色宿缩_搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆=笔骨北得谷福曲_搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆=吉竹菊足烛_搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆=色宿缩_铁雪血脚百客尺=黑织哭_铁雪血脚百客尺=吸息积惜击_铁雪血脚百客尺=吉竹菊足烛_铁雪血脚百客尺=七虱一出叔', weight=-0.3603347156074595, std=None, value=None)], pos_remaining=75, neg_remaining=28), proba=None, score=None, weighted_spans=None, heatmap=None), TargetExplanation(target='中原官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=立入_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=落药麦_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=业灭列孽末越弱_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩

In [8]:
train_feature = train_data[train_dialect != '']
test_feature = test_data[test_dialect != '']
train_target = train_dialect[train_dialect != ''] == '闽方言'
test_target = test_dialect[test_dialect != ''] == '闽方言'

In [9]:
svm = SVC(kernel='linear').fit(train_feature, train_target)
print(classification_report(train_target, svm.predict(train_feature)))
print(classification_report(test_target, svm.predict(test_feature)))
eli5.explain_weights(svm, top=20, feature_names=train_data.columns.values)

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       589
        True       1.00      1.00      1.00        53

    accuracy                           1.00       642
   macro avg       1.00      1.00      1.00       642
weighted avg       1.00      1.00      1.00       642

              precision    recall  f1-score   support

       False       1.00      0.99      1.00       154
        True       0.89      1.00      0.94         8

    accuracy                           0.99       162
   macro avg       0.94      1.00      0.97       162
weighted avg       0.99      0.99      0.99       162



Explanation(estimator="SVC(kernel='linear')", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=True, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.09432960624200176, std=None, value=None), FeatureWeight(feature='initial_意衣腰要优幼厌音烟印一隐秧约影益拥=爱暗安恩恶', weight=0.07092198962219559, std=None, value=None), FeatureWeight(feature='initial_举句卷决均橘军菊=轿舅旧近_坐罪字杂=左租做紫资子早灶卒作粽_轿舅旧近=鸡寄饥记几季_轿舅旧近=假嫁锯交减监甲奸江讲_轿舅旧近=叫九剑金急建肩见结紧吉筋劲姜脚镜经击', weight=0.059264820582476424, std=None, value=None), FeatureWeight(feature='initial_把布贝拜摆杯碑比宝包饱表扮八班板变扁憋搬半拨笔本帮绑剥北冰逼百兵柄饼壁=缚', weight=0.05804348193402828, std=None, value=None), FeatureWeight(feature='initial_多躲赌带低对刀钓抖丢搭胆点店跌单典端短墩蹲党灯等凳得打钉顶东懂冻冬=大袋弟递地道豆淡碟垫断夺定笛动洞读毒_把布贝拜摆杯碑比宝包饱表扮八班板变扁憋搬半拨笔本帮绑剥北冰逼百兵柄饼壁=步败币背办便别薄棒白病', weight=0.05457509946299047, std=None, value=None), FeatureWeight(feature='tone_左可躲锁果火把假哑姐写谱赌土古苦虎所鼠举许府取数主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝早草嫂好饱找小抖走狗口酒手九感胆毯敢减险点品铲产板剪浅扁典显短碗选卷反紧隐本滚笋准粉党抢想厂响纺绑讲等肯打省梗影饼井整顶挺懂桶孔统肿=有懒痒', weight=0.05418673942485659, std=None, value=None), FeatureWeight(feature='initial_多躲赌带低对刀钓抖丢搭胆点店跌单典端短墩蹲党灯等凳得打钉顶东懂冻冬=图台条甜弹田藤铜_把布贝拜摆杯碑比宝包饱表扮八班板变扁憋搬半拨笔本帮绑剥北冰逼百兵柄饼壁=婆爬排牌赔皮盘贫盆朋棚平瓶蓬', weight=0.05198761689122988, std=None, value=None), FeatureWeight(feature='initial_府付富副福=滑还横_父肥浮妇犯饭罚坟佛房防凤服缝=滑还横_肺飞费法反翻发分粉粪方放纺风丰封蜂=滑还横', weight=0.051867273013721134, std=None, value=None), FeatureWeight(feature='tone_延顽=虾_河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容=虾_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓=虾_斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停=虾_虾=螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙_虾=茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷', weight=0.050582706785193644, std=None, value=None)], neg=[FeatureWeight(feature='initial_嫌协现县形熊雄=鞋咸项', weight=-0.05965067381389384, std=None, value=None), FeatureWeight(feature='tone_会问让=祸_夏汗换县闰=祸_饿磨夜路卖外二帽闹庙料右念烂岸慢面乱认嫩浪亮样硬命梦用=祸_父罪道造限善件静=祸_遇艺义类位胃验院运旺=祸_大谢射步雾住树袋害败坏画话地字事柜味号轿豆寿旧办便垫现饭万顺剩病定洞共=祸_校袖任恨凤=祸', weight=-0.05660045237302319, std=None, value=None), FeatureWeight(feature='finals_抽绸州臭手寿=抖偷头豆楼走钩狗够口藕后厚', weight=-0.0559226368457047, std=None, value=None), FeatureWeight(feature='finals_灯等凳藤能层僧=分粉粪坟', weight=-0.055770896374432534, std=None, value=None), FeatureWeight(feature='initial_许系戏喜希=鞋咸项_鞋咸项=靴孝休险歇显血熏响向兴兄凶', weight=-0.05432754920657998, std=None, value=None), FeatureWeight(feature='initial_步败币背办便别薄棒白病=被抱', weight=-0.052567948943976886, std=None, value=None), FeatureWeight(feature='finals_本盆门=灯等凳藤能层僧', weight=-0.05251622704631666, std=None, value=None), FeatureWeight(feature='finals_贞=证秤绳剩升程整正声城', weight=-0.05239738087008629, std=None, value=None), FeatureWeight(feature='finals_本盆门=根恨恩肯_根恨恩肯=分粉粪坟', weight=-0.051857269293602265, std=None, value=None), FeatureWeight(feature='finals_方放纺房防=帮忙党汤糖浪仓钢糠绑胖棒', weight=-0.05167637691011793, std=None, value=None), FeatureWeight(feature='initial_乌屋=滑还横_歪卫位围胃完碗挖弯温王旺握翁=滑还横', weight=-0.05146339435423815, std=None, value=None)], pos_remaining=1352, neg_remaining=1629), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [10]:
train_feature = train_data[train_dialect != '']
test_feature = test_data[test_dialect != '']
train_target = train_dialect[train_dialect != ''].str.endswith('官话')
test_target = test_dialect[test_dialect != ''].str.endswith('官话')

In [11]:
selector = SelectKBest(chi2, k=1000).fit(train_feature, train_target)
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_feature.iloc[:, idx],
    train_feature.index,
    columns=train_feature.columns[idx]
)
df.insert(0, 'label', train_target)
df.insert(1, 'dialect', train_dialect[train_dialect != ''])
df.insert(2, 'location', location.loc[train_feature.index, ['province', 'city', 'country']].fillna('').apply(lambda x: ','.join(x), axis=1))
dtale.show(df, name='guanhua chi2')

In [12]:
svm = SVC(kernel='linear').fit(train_feature, train_target)
print(classification_report(train_target, svm.predict(train_feature)))
print(classification_report(test_target, svm.predict(test_feature)))
eli5.explain_weights(svm, top=20, feature_names=train_data.columns.values)

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       314
        True       1.00      1.00      1.00       328

    accuracy                           1.00       642
   macro avg       1.00      1.00      1.00       642
weighted avg       1.00      1.00      1.00       642

              precision    recall  f1-score   support

       False       0.94      0.96      0.95        71
        True       0.97      0.96      0.96        91

    accuracy                           0.96       162
   macro avg       0.96      0.96      0.96       162
weighted avg       0.96      0.96      0.96       162



Weight?,Feature
+0.202,initial_把布贝拜摆杯碑比宝包饱表扮八班板变扁憋搬半拨笔本帮绑剥北冰逼百兵柄饼壁=被抱
+0.179,tone_河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容=渠_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓=渠_斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停=渠_螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙=渠_茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷=渠
+0.147,initial_错菜刺草糙凑擦村寸仓葱=侧测策
+0.137,tone_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=接贴切泼脱缺踢吃_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=黑织哭_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=吸息积惜击_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=七虱一出叔
+0.132,finals_夹甲鸭瞎=敲_假嫁牙虾下夏哑=敲
+0.127,initial_该改梗=柜共_柜共=歌个果过瓜古盖怪拐挂桂规龟鬼贵高钩狗够感鸽甘敢肝割官关惯刮根滚骨钢各光郭国更格耕隔公谷宫恭
+0.120,tone_或=河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容_或=锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓_或=斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停_或=螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙_或=茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷
+0.117,initial_茶柴=侧测策
+0.117,tone_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=刻侧测_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=笔骨北得谷福曲_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=铁雪血脚百客尺_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=作各恶策_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=折格隔_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=节结国_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=吉竹菊足烛_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=甲法_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=色宿缩_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=塔渴
+0.116,finals_把爬马骂茶沙打=绕_法发罚=绕_搭踏杂塔蜡插闸=绕_达辣擦八扎杀=绕_绕=大拉


In [13]:
train_feature = train_data[train_dialect.isin(['赣方言', '客家方言'])]
test_feature = test_data[test_dialect.isin(['赣方言', '客家方言'])]
train_target = train_dialect[train_dialect.isin(['赣方言', '客家方言'])] == '客家方言'
test_target = test_dialect[test_dialect.isin(['赣方言', '客家方言'])] == '客家方言'

In [14]:
selector = SelectKBest(chi2, k=1000).fit(train_feature, train_target)
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_feature.iloc[:, idx],
    index=train_feature.index,
    columns=train_feature.columns[idx]
)
df.insert(0, 'label', train_target)
df.insert(1, 'dialect', train_dialect[train_dialect.isin(['赣方言', '客家方言'])])
df.insert(2, 'location', location.loc[train_feature.index, ['province', 'city', 'country']].fillna('').apply(lambda x: ','.join(x), axis=1))
dtale.show(df, name='kejia chi2')

In [15]:
svm = SVC(kernel='linear').fit(train_feature, train_target)
print(classification_report(train_target, svm.predict(train_feature)))
print(classification_report(test_target, svm.predict(test_feature)))
eli5.explain_weights(svm, top=10, feature_names=train_data.columns.values)

              precision    recall  f1-score   support

       False       1.00      1.00      1.00        57
        True       1.00      1.00      1.00        34

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91

              precision    recall  f1-score   support

       False       1.00      0.92      0.96        12
        True       0.83      1.00      0.91         5

    accuracy                           0.94        17
   macro avg       0.92      0.96      0.93        17
weighted avg       0.95      0.94      0.94        17



Explanation(estimator="SVC(kernel='linear')", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=True, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='initial_余雨芋裕移姨=哑鸭_圆院冤园远越匀云运永=哑鸭', weight=0.05479611499210272, std=None, value=None), FeatureWeight(feature='initial_意衣腰要优幼厌音烟印一隐秧约影益拥=哑鸭_爷野夜摇有右油炎盐叶延引痒样药蝇赢营育用浴=哑鸭', weight=0.05302538898654416, std=None, value=None), FeatureWeight(feature='finals_墩嫩村寸蹲孙轮笋=吞', weight=0.04557291257948203, std=None, value=None)], neg=[FeatureWeight(feature='initial_锣螺路来雷类老楼拉蓝蜡兰懒烂辣乱轮浪落冷聋鹿六龙绿=奴脑闹南难暖嫩能脓浓_奴脑闹南难暖嫩能脓浓=吕犁梨李料流林立连列莲邻栗律两亮力领零历', weight=-0.07239877552075957, std=None, value=None), FeatureWeight(feature='<BIAS>', weight=-0.06491404925072963, std=None, value=None), FeatureWeight(feature='tone_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=猫', weight=-0.058909887396158905, std=None, value=None), FeatureWeight(feature='finals_桂规亏跪龟柜鬼贵=卫危位围胃', weight=-0.05291269760125808, std=None, value=None), FeatureWeight(feature='initial_写西洗小笑箫修心鲜线先选雪新想削息姓惜星锡=吸', weight=-0.05122817742560219, std=None, value=None), FeatureWeight(feature='finals_女吕徐举锯渠鱼许余取句区遇雨芋裕剧玉=岁嘴随醉', weight=-0.049977373450320564, std=None, value=None), FeatureWeight(feature='initial_锁所死四丝嫂三伞酸算孙笋索塞送松宋缩=熟赎属', weight=-0.04597111196128169, std=None, value=None)], pos_remaining=1574, neg_remaining=1414), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [16]:
dt = DecisionTreeClassifier(max_depth=1).fit(train_feature, train_target)
print(classification_report(train_target, dt.predict(train_feature)))
print(classification_report(test_target, dt.predict(test_feature)))
eli5.explain_weights(dt, top=10, feature_names=train_data.columns.values)

              precision    recall  f1-score   support

       False       0.92      0.98      0.95        57
        True       0.97      0.85      0.91        34

    accuracy                           0.93        91
   macro avg       0.94      0.92      0.93        91
weighted avg       0.94      0.93      0.93        91

              precision    recall  f1-score   support

       False       0.92      0.92      0.92        12
        True       0.80      0.80      0.80         5

    accuracy                           0.88        17
   macro avg       0.86      0.86      0.86        17
weighted avg       0.88      0.88      0.88        17



Weight,Feature
1.0000,tone_角=业灭列孽末越弱_立入=甲法_失=业灭列孽末越弱_业灭列孽末越弱=窄_业灭列孽末越弱=鸽_业灭列孽末越弱=塞_业灭列孽末越弱=锡_业灭列孽末越弱=橘_密墨力鹿绿=锡_蜡叶辣热月=锡
0,initial_罩找闸扎争=侄着直_抓装壮桩桌=侄着直_中终竹肿种烛=侄着直
0,initial_坐罪字杂=镯_走=镯_造=镯_镯=贼_镯=族_镯=嘴醉
0,initial_制照占折战张章粥=制照占折战张章粥_制照占折战张章粥=州针汁震证织整正_车抽臭撤厂唱秤=车抽臭撤厂唱秤_州针汁震证织整正=州针汁震证织整正
0,initial_辰尝=辰尝_矮=爱暗安恩恶
0,initial_镯=侄着直
0,finals_江讲降=网_网=项
0,initial_乌屋=花化灰欢婚慌_许系戏喜希=余雨芋裕移姨_余雨芋裕移姨=花化灰欢婚慌_余雨芋裕移姨=海好喊汉黑烘_余雨芋裕移姨=靴孝休险歇显血熏响向兴兄凶_余雨芋裕移姨=火货霍_歪卫位围胃完碗挖弯温王旺握翁=花化灰欢婚慌_歪卫位围胃完碗挖弯温王旺握翁=靴孝休险歇显血熏响向兴兄凶_花化灰欢婚慌=意衣腰要优幼厌音烟印一隐秧约影益拥_花化灰欢婚慌=圆院冤园远越匀云运永_花化灰欢婚慌=爷野夜摇有右油炎盐叶延引痒样药蝇赢营育用浴_花化灰欢婚慌=哑鸭_花化灰欢婚慌=荣容_花化灰欢婚慌=爱暗安恩恶_意衣腰要优幼厌音烟印一隐秧约影益拥=靴孝休险歇显血熏响向兴兄凶_话换活黄=靴孝休险歇显血熏响向兴兄凶_圆院冤园远越匀云运永=靴孝休险歇显血熏响向兴兄凶_爷野夜摇有右油炎盐叶延引痒样药蝇赢营育用浴=靴孝休险歇显血熏响向兴兄凶_靴孝休险歇显血熏响向兴兄凶=爱暗安恩恶
0,finals_破婆磨磨摸=或_或=拨泼末_或=鹅饿_或=鸽盒_或=歌个可河课_或=割渴
0,tone_借晒瘦=缚_玉=缚_式=缚_震=缚_糙=缚_缚=剧_缚=契_缚=扮_缚=个_缚=做_缚=胖_缚=兴_缚=付_缚=错


In [17]:
train_feature = train_data[(train_dialect == '晋方言') | train_dialect.str.endswith('官话')]
test_feature = test_data[(test_dialect == '晋方言') | test_dialect.str.endswith('官话')]
train_target = train_dialect[(train_dialect == '晋方言') | train_dialect.str.endswith('官话')] == '晋方言'
test_target = test_dialect[(test_dialect == '晋方言') | test_dialect.str.endswith('官话')] == '晋方言'

In [18]:
selector = SelectKBest(chi2, k=1000).fit(train_feature, train_target)
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_feature.iloc[:, idx],
    index=train_feature.index,
    columns=train_feature.columns[idx]
)
df.insert(0, 'label', train_target)
df.insert(1, 'dialect', train_dialect[train_dialect != ''])
df.insert(2, 'location', location.loc[train_feature.index, ['province', 'city', 'country']].fillna('').apply(lambda x: ','.join(x), axis=1))
dtale.show(df, name='jin chi2')

In [19]:
svm = SVC(kernel='linear').fit(train_feature, train_target)
print(classification_report(train_target, svm.predict(train_feature)))
print(classification_report(test_target, svm.predict(test_feature)))
eli5.explain_weights(svm, top=20, feature_names=train_data.columns.values)

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       328
        True       1.00      1.00      1.00        35

    accuracy                           1.00       363
   macro avg       1.00      1.00      1.00       363
weighted avg       1.00      1.00      1.00       363

              precision    recall  f1-score   support

       False       1.00      1.00      1.00        91
        True       1.00      1.00      1.00         9

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



Explanation(estimator="SVC(kernel='linear')", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=True, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_延顽=虾_河华如余回怀随浮船圆神辰坟房防降绳行形荣红熊雄容=虾_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝颜棉年完人门匀云忙迎名赢零脓=虾_斜图徐除锄台池骑迟祠垂朝愁球含甜嫌寻琴弹还全传权贫勤唇纯长尝狂朋层程城停=虾_虾=螺奴吴煤时楼流岩严林难兰连言莲还原园民邻银轮娘黄王能明营浓龙_虾=茄婆爬茶蛇壶财排牌柴鞋赔皮棋锤肥桃桥条头绸潭蚕咸钳沉钱缠田前盘陈盆魂裙糖床藤棚平瓶蓬铜虫穷', weight=0.06981583283801572, std=None, value=None), FeatureWeight(feature='initial_姐借焦酒尖接剪节进俊浆井积=足', weight=0.059178099437047035, std=None, value=None), FeatureWeight(feature='finals_宫龙松恭共=滚困婚魂温_东懂冻通桶痛铜动洞聋弄粽葱送公孔烘红翁冬统脓松宋=滚困婚魂温_滚困婚魂温=中虫终充浓重肿种冲', weight=0.055322792696258734, std=None, value=None), FeatureWeight(feature='initial_姐借焦酒尖接剪节进俊浆井积=浸', weight=0.05445599083777086, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.48122803057006436, std=None, value=None), FeatureWeight(feature='tone_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=接贴切泼脱缺踢吃_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=搭插鸭擦割八扎杀瞎歇拨刷刮托削郭剥桌拍拆_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=黑织哭_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=吸息积惜击_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=七虱一出叔', weight=-0.10405692635129954, std=None, value=None), FeatureWeight(feature='finals_贝杯配赔背煤妹碑=被眉', weight=-0.07747932638492093, std=None, value=None), FeatureWeight(feature='tone_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=刻侧测_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=笔骨北得谷福曲_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=铁雪血脚百客尺_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=作各恶策_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=折格隔_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=节结国_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=吉竹菊足烛_多拖歌靴沙车瓜花铺租箍乌猪初书输区胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢闩关弯砖圈翻冤吞根恩亲新身筋墩村蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方筐桩窗双江灯僧冰升生更坑争耕兵清贞声轻钉厅青星经兄东通葱公翁冬松风丰中终充宫封蜂冲恭凶=甲

In [20]:
train_feature = train_data[train_dialect.isin(['粤方言', '平话'])]
test_feature = test_data[test_dialect.isin(['粤方言', '平话'])]
train_target = train_dialect[train_dialect.isin(['粤方言', '平话'])] == '平话'
test_target = test_dialect[test_dialect.isin(['粤方言', '平话'])] == '平话'

In [21]:
svm = SVC(kernel='linear').fit(train_feature, train_target)
print(classification_report(train_target, svm.predict(train_feature)))
print(classification_report(test_target, svm.predict(test_feature)))
eli5.explain_weights(svm, top=20, feature_names=train_data.columns.values)

              precision    recall  f1-score   support

       False       1.00      1.00      1.00        38
        True       1.00      1.00      1.00         9

    accuracy                           1.00        47
   macro avg       1.00      1.00      1.00        47
weighted avg       1.00      1.00      1.00        47

              precision    recall  f1-score   support

       False       0.89      1.00      0.94         8
        True       1.00      0.50      0.67         2

    accuracy                           0.90        10
   macro avg       0.94      0.75      0.80        10
weighted avg       0.91      0.90      0.89        10



Explanation(estimator="SVC(kernel='linear')", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=True, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1.066162234162142, std=None, value=None), FeatureWeight(feature='initial_歪卫位围胃完碗挖弯温王旺握翁=瓦外', weight=0.05126924468151793, std=None, value=None), FeatureWeight(feature='initial_海好喊汉黑烘=火货霍', weight=0.04886121999042472, std=None, value=None), FeatureWeight(feature='tone_借晒瘦=糙_破课裤去菜派剃配快刺屁器气靠炮票凑臭欠炭看骗片判劝寸困唱秤庆痛=糙_糙=过货嫁化布锯数句爱贝带盖拜戒制世对碎怪挂岁肺桂寄戏四试记意醉季费贵灶罩孝笑照要钓叫够富副幼暗占厌剑店汉变线战扇建见半算惯进镇印劲俊粪壮向放凳证镜姓正冻粽送宋种', weight=0.046255960056374515, std=None, value=None), FeatureWeight(feature='initial_奴脑闹南难暖嫩能脓浓=验严业', weight=0.0443527861740244, std=None, value=None), FeatureWeight(feature='initial_验严业=泥牛黏念孽年捏娘', weight=0.03855265470588107, std=None, value=None), FeatureWeight(feature='finals_证秤绳剩升程整正声城=兴蝇行庆迎静经形', weight=0.0385244669385624, std=None, value=None)], neg=[FeatureWeight(feature='finals_胎台袋来菜财该改开海爱盖害=外', weight=-0.05026136396998811, std=None, value=None), FeatureWeight(feature='finals_把爬马骂茶沙打=瓦_假嫁牙虾下夏哑=瓦', weight=-0.04635996516306694, std=None, value=None), FeatureWeight(feature='finals_端短断暖乱酸算=肝看岸汉汗安', weight=-0.045934547063150366, std=None, value=None), FeatureWeight(feature='finals_官宽欢换碗=肝看岸汉汗安', weight=-0.04561602626109763, std=None, value=None), FeatureWeight(feature='tone_左可躲锁果火把假哑姐写谱赌土古苦虎所鼠举许府取数主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝早草嫂好饱找小抖走狗口酒手九感胆毯敢减险点品铲产板剪浅扁典显短碗选卷反紧隐本滚笋准粉党抢想厂响纺绑讲等肯打省梗影饼井整顶挺懂桶孔统肿=蟹_有懒痒=蟹_女武米李脑染眼晚引=蟹_蟹=马野吕买尾母满暖软两猛冷领_蟹=鸟永_蟹=瓦雨耳老藕远网', weight=-0.04234995838773953, std=None, value=None), FeatureWeight(feature='finals_装壮疮床霜桩撞窗双=帮忙党汤糖浪仓钢糠绑胖棒', weight=-0.04186845436132164, std=None, value=None), FeatureWeight(feature='finals_搬半判盘满=肝看岸汉汗安', weight=-0.040690731737145616, std=None, value=None), FeatureWeight(feature='tone_夏汗换县闰=下弟动_饿磨夜路卖外二帽闹庙料右念烂岸慢面乱认嫩浪亮样硬命梦用=下弟动_遇艺义类位胃验院运旺=下弟动_大谢射步雾住树袋害败坏画话地字事柜味号轿豆寿旧办便垫现饭万顺剩病定洞共=下弟动_下弟动=校袖任恨凤', weight=-0.0405528015276731, std=None, value=None), FeatureWeight(feature='finals_端短断暖乱酸算=转传传砖船软', weight=-0.039681181084587014, std=None, value=None), FeatureWeight(feature='tone_会问让=币_夏汗换县闰=币_饿磨夜路卖外二帽闹庙料右念烂岸慢面乱认嫩浪亮样硬命梦用=币_父罪道造限善件静=币_遇艺义类位胃验院运旺=币_大谢射步雾住树袋害败坏画话地字事柜味号轿豆寿旧办便垫现饭万顺剩病定洞共=币_校袖任恨凤=币', weight=-0.03904878716519897, std=None, value=None), FeatureWeight(feature='initial_府付富副福=花化灰欢婚慌_父肥浮妇犯饭罚坟佛房防凤服缝=花化灰欢婚慌_花化灰欢婚慌=肺飞费法反翻发分粉粪方放纺风丰封蜂', weight=-0.03883859723343345, std=None, value=None), FeatureWeight(feature='initial_许系戏喜希=校_靴孝休险歇显血熏响向兴兄凶=校', weight=-0.0381424317111071, std=None, value=None), FeatureWeight(feature='initial_厚含汗=校_河害号后盒还恨鹤红=校_海好喊汉黑烘=校', weight=-0.03799446454977148, std=None, value=None)], pos_remaining=1284, neg_remaining=1677), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)